In [ ]:
%load_ext rpy2.ipython
from datetime import datetime
from matplotlib import colors
from matplotlib import pyplot as plt
from rpy2.robjects import r, pandas2ri
import rpy2.robjects as ro
import numpy as np
import pandas as pd


In [ ]:
%%R
library("NHPoisson")

In [ ]:
df = pd.read_csv('/directory/file_name.csv')
BEHAPP_ID_list = df['ID'].unique()
pd.DataFrame.iteritems = pd.DataFrame.items  # Do this since our pandas version doesn't have iteritems and causes issue with rpy2

## Functions

In [ ]:
# For each participant set each 10% of data to different masking for cross-validation
def mask_data_cross_validation(dataframe,BEHAPP_ID_list):
    locations = []
    df['Mask'] = np.nan
    df.sort_values(by=['ID', 'hour'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    for ID in BEHAPP_ID_list:
        start_index = df.loc[df['ID'] == ID].index[0]
        participant_end_index = df.loc[df['ID'] == ID].index[-1]
        length_current_id = len(df[df['ID'] == ID])    
        length_each_masked_section = round(0.1*length_current_id)
        end_index = start_index + length_each_masked_section
        mask = 1 # Start mask value at 1
        for i in range(10):
            if end_index > participant_end_index:
                end_index = participant_end_index
            df.loc[start_index:end_index, 'Mask'] = mask 
            start_index = end_index
            end_index = end_index + length_each_masked_section  
            mask = mask + 1
        df.loc[df['Mask'].isna(), 'Mask'] = 10  # Any non-assigned because of rounding added to last
    return df    

# Bin data for all participants: activity set to 2, no activity set to 1
def bin_data_cross_validation(dataframe):
    df['Social media App'] = df['Social media App'].apply(lambda x: 2 if x>0 else 1)
    df['Communication App'] = df['Communication App'].apply(lambda x: 2 if x>0 else 1)
    df['Phone Usage'] = df['Phone Usage'].apply(lambda x: 2 if x>0 else 1)
    df['Traveling'] = df['Traveling'].apply(lambda x: 2 if x>0 else 1)
    df['Stationary'] = df['Stationary'].apply(lambda x: 3 if x==1 else 1 if x==0 else 2)
    df['Home stay'] = df['Home stay'].apply(lambda x: 3 if x==1 else 1 if x==0 else 2)
    df_original = df.copy()
    
    #  If has_data is 0, set all other columns to -1 (do 2nd so can see what masking was already missing)
    df.loc[df['has_data'] == 0, ['Social media App', 'Communication App', 'Phone Usage', 'Traveling', 'Stationary', 'Home stay']] = -1
    #  If has_gps_data is 0, set all GPS columns to -1
    df.loc[df['has_gps_data'] == 0, ['Traveling', 'Stationary', 'Home stay']] = -1
    return df, df_original

# Covariate handling
def sin_cos_transform(value, period):
    x = np.sin(value / period * 2 * np.pi)
    y = np.cos(value / period * 2 * np.pi)
    return np.array([x,y])

def encode_covariates(dataframe,channel,encode):
    df = dataframe
    df_ID_app = df[['ID','weekday','hour',channel,'Mask']].copy()
    # Hour
    df_ID_app['hour'] = pd.to_datetime(df_ID_app['hour'], format='%Y-%m-%d %H:%M:%S').dt.hour
    # Day
    df_ID_app.loc[df_ID_app['weekday']=='Sunday', 'weekday'] = 0
    df_ID_app.loc[df_ID_app['weekday']=='Monday', 'weekday'] = 1
    df_ID_app.loc[df_ID_app['weekday']=='Tuesday', 'weekday'] = 2
    df_ID_app.loc[df_ID_app['weekday']=='Wednesday', 'weekday'] = 3
    df_ID_app.loc[df_ID_app['weekday']=='Thursday', 'weekday'] = 4
    df_ID_app.loc[df_ID_app['weekday']=='Friday', 'weekday'] = 5
    df_ID_app.loc[df_ID_app['weekday']=='Saturday', 'weekday'] = 6
    if encode=="sin_cos":
        df_ID_app['sin_hour'] = df_ID_app['hour'].apply(lambda x: sin_cos_transform(x,24)[0])
        df_ID_app['cos_hour'] = df_ID_app['hour'].apply(lambda x: sin_cos_transform(x,24)[1])
        df_ID_app['sin_weekday'] = df_ID_app['weekday'].apply(lambda x: sin_cos_transform(x,7)[0])
        df_ID_app['cos_weekday'] = df_ID_app['weekday'].apply(lambda x: sin_cos_transform(x,7)[1])
    if encode=="onehot":
        df_ID_app = df_ID_app.sort_values(by=['weekday', 'hour'])  # Sorting to make sure days and hours always get same assignment
        onehot_hour = pd.get_dummies(df_ID_app['hour'], dtype=float, prefix="onehot_hour", drop_first=True)
        onehot_weekday = pd.get_dummies(df_ID_app['weekday'], dtype=float, prefix="onehot_weekday", drop_first=True)
        df_ID_app = pd.concat([df_ID_app, onehot_hour,  onehot_weekday], axis=1)
        df_ID_app = df_ID_app.sort_index()  # Re-sort by index
                  
    return df_ID_app


In [ ]:
# Cycles needed for simulation
# Sin cos cycles
basic_sin_day_cycle = [sin_cos_transform(x,7)[0] for x in range(7)]  
basic_cos_day_cycle = [sin_cos_transform(x,7)[1] for x in range(7)]  
basic_sin_hour_cycle = [sin_cos_transform(x,24)[0] for x in range(24)]  
basic_cos_hour_cycle = [sin_cos_transform(x,24)[1] for x in range(24)]  
# One-hot encoded cycles
basic_onehot_day_matrix_cycle = np.zeros(shape=(6,6))
np.fill_diagonal(basic_onehot_day_matrix_cycle, 1)
basic_onehot_day_matrix_cycle = np.append(np.zeros(shape=(1,6)), basic_onehot_day_matrix_cycle, axis=0)
basic_onehot_hour_matrix_cycle = np.zeros(shape=(23,23))
np.fill_diagonal(basic_onehot_hour_matrix_cycle, 1)
basic_onehot_hour_matrix_cycle = np.append(np.zeros(shape=(1,23)), basic_onehot_hour_matrix_cycle, axis=0)

In [ ]:
%%R
# 1 HOT ENCODED
model_training_onehot <- function(channel_name){
    channel <- channel_name  # 'Phone Usage'  # 'Communication App'  # 'Social media App'
    for (fold in 1:10) {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), logLik_intercept_day=double(), coefficients_intercept_day_b0=double(), coefficients_intercept_day_b1=double(), coefficients_intercept_day_b2=double(), coefficients_intercept_day_b3=double(), coefficients_intercept_day_b4=double(), coefficients_intercept_day_b5=double(), coefficients_intercept_day_b6=double(), logLik_intercept_hour=double(), coefficients_intercept_hour_b0=double(), coefficients_intercept_hour_b1=double(), coefficients_intercept_hour_b2=double(), coefficients_intercept_hour_b3=double(), coefficients_intercept_hour_b4=double(), coefficients_intercept_hour_b5=double(), coefficients_intercept_hour_b6=double(), coefficients_intercept_hour_b7=double(), coefficients_intercept_hour_b8=double(), coefficients_intercept_hour_b9=double(), coefficients_intercept_hour_b10=double(), coefficients_intercept_hour_b11=double(), coefficients_intercept_hour_b12=double(), coefficients_intercept_hour_b13=double(), coefficients_intercept_hour_b14=double(), coefficients_intercept_hour_b15=double(), coefficients_intercept_hour_b16=double(), coefficients_intercept_hour_b17=double(), coefficients_intercept_hour_b18=double(), coefficients_intercept_hour_b19=double(), coefficients_intercept_hour_b20=double(), coefficients_intercept_hour_b21=double(), coefficients_intercept_hour_b22=double(), coefficients_intercept_hour_b23=double()) 
        df_likelihood_comparison <- data.frame('Fold'=character(), 'ID'=character(), 'Day_likelihood'=double(), 'Hour_likelihood'=double())
        for (ID in BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]  # df with masking
            df_current <- df_current[df_current$Mask != fold, ]
            timestamps <- which(df_current[channel] == 2)
            if (length(timestamps)==0){
                # With day
                logLik_intercept_day <- NA
                coefficients_intercept_day_b0 <- NA
                coefficients_intercept_day_b1 <- NA
                coefficients_intercept_day_b2 <- NA
                coefficients_intercept_day_b3 <- NA
                coefficients_intercept_day_b4 <- NA
                coefficients_intercept_day_b5 <- NA
                coefficients_intercept_day_b6 <- NA
                testlik_day <- list(pv=NA)
               
                # With hour
                logLik_intercept_hour <- NA
                coefficients_intercept_hour_b0 <- NA
                coefficients_intercept_hour_b1 <- NA
                coefficients_intercept_hour_b2 <- NA
                coefficients_intercept_hour_b3 <- NA
                coefficients_intercept_hour_b4 <- NA
                coefficients_intercept_hour_b5 <- NA
                coefficients_intercept_hour_b6 <- NA
                coefficients_intercept_hour_b7 <- NA
                coefficients_intercept_hour_b8 <- NA
                coefficients_intercept_hour_b9 <- NA
                coefficients_intercept_hour_b10 <- NA
                coefficients_intercept_hour_b11 <- NA
                coefficients_intercept_hour_b12 <- NA
                coefficients_intercept_hour_b13 <- NA
                coefficients_intercept_hour_b14 <- NA
                coefficients_intercept_hour_b15 <- NA
                coefficients_intercept_hour_b16 <- NA
                coefficients_intercept_hour_b17 <- NA
                coefficients_intercept_hour_b18 <- NA
                coefficients_intercept_hour_b19 <- NA
                coefficients_intercept_hour_b20 <- NA
                coefficients_intercept_hour_b21 <- NA
                coefficients_intercept_hour_b22 <- NA
                coefficients_intercept_hour_b23 <- NA
                testlik_hour <- list(pv=NA)
                
            } else {
                model_homog <- fitPP.fun(covariates=NULL, posE=timestamps, nobs=length(df_current$weekday), start=list(b0=0))
                
                model_day <- try(fitPP.fun(covariates=cbind(df_current$onehot_weekday_1,df_current$onehot_weekday_2,df_current$onehot_weekday_3,df_current$onehot_weekday_4,df_current$onehot_weekday_5,df_current$onehot_weekday_6), posE=timestamps, nobs=length(df_current$weekday), start=list(b0=0,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0)))
                if (inherits(model_day, 'try-error')){
                    logLik_intercept_day <- 'Failed'
                    coefficients_intercept_day_b0 <- 'Failed'
                    coefficients_intercept_day_b1 <- 'Failed'
                    coefficients_intercept_day_b2 <- 'Failed'
                    coefficients_intercept_day_b3 <- 'Failed'
                    coefficients_intercept_day_b4 <- 'Failed'
                    coefficients_intercept_day_b5 <- 'Failed'
                    coefficients_intercept_day_b6 <- 'Failed'
                    testlik_day <- list(pv='Failed')
                }   
                else {
                    logLik_intercept_day <- logLik(model_day)
                    coefficients_intercept_day_b0 <- t(coef(model_day))[1,][1]
                    coefficients_intercept_day_b1 <- t(coef(model_day))[1,][2]
                    coefficients_intercept_day_b2 <- t(coef(model_day))[1,][3]
                    coefficients_intercept_day_b3 <- t(coef(model_day))[1,][4]
                    coefficients_intercept_day_b4 <- t(coef(model_day))[1,][5]
                    coefficients_intercept_day_b5 <- t(coef(model_day))[1,][6]
                    coefficients_intercept_day_b6 <- t(coef(model_day))[1,][7]
                    testlik_day <- testlik.fun(model_day, model_homog)
                }
                
                model_hour <- try(fitPP.fun(covariates=cbind(df_current$onehot_hour_1,df_current$onehot_hour_2,df_current$onehot_hour_3,df_current$onehot_hour_4,df_current$onehot_hour_5,df_current$onehot_hour_6,df_current$onehot_hour_7,df_current$onehot_hour_8,df_current$onehot_hour_9,df_current$onehot_hour_10,df_current$onehot_hour_11,df_current$onehot_hour_12,df_current$onehot_hour_13,df_current$onehot_hour_14,df_current$onehot_hour_15,df_current$onehot_hour_16,df_current$onehot_hour_17,df_current$onehot_hour_18,df_current$onehot_hour_19,df_current$onehot_hour_20,df_current$onehot_hour_21,df_current$onehot_hour_22,df_current$onehot_hour_23), posE=timestamps, nobs=length(df_current$hour), start=list(b0=0,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0,b7=0,b8=0,b9=0,b10=0,b11=0,b12=0,b13=0,b14=0,b15=0,b16=0,b17=0,b18=0,b19=0,b20=0,b21=0,b22=0,b23=0),lambdaylim=c(-1000,1000)))
                if (inherits(model_hour, 'try-error')){
                    logLik_intercept_hour <- 'Failed'
                    coefficients_intercept_hour_b0 <- 'Failed'
                    coefficients_intercept_hour_b1 <- 'Failed'
                    coefficients_intercept_hour_b2 <- 'Failed'
                    coefficients_intercept_hour_b3 <- 'Failed'
                    coefficients_intercept_hour_b4 <- 'Failed'
                    coefficients_intercept_hour_b5 <- 'Failed'
                    coefficients_intercept_hour_b6 <- 'Failed'
                    coefficients_intercept_hour_b7 <- 'Failed'
                    coefficients_intercept_hour_b8 <- 'Failed'
                    coefficients_intercept_hour_b9 <- 'Failed'
                    coefficients_intercept_hour_b10 <- 'Failed'
                    coefficients_intercept_hour_b11 <- 'Failed'
                    coefficients_intercept_hour_b12 <- 'Failed'
                    coefficients_intercept_hour_b13 <- 'Failed'
                    coefficients_intercept_hour_b14 <- 'Failed'
                    coefficients_intercept_hour_b15 <- 'Failed'
                    coefficients_intercept_hour_b16 <- 'Failed'
                    coefficients_intercept_hour_b17 <- 'Failed'
                    coefficients_intercept_hour_b18 <- 'Failed'
                    coefficients_intercept_hour_b19 <- 'Failed'
                    coefficients_intercept_hour_b20 <- 'Failed'
                    coefficients_intercept_hour_b21 <- 'Failed'
                    coefficients_intercept_hour_b22 <- 'Failed'
                    coefficients_intercept_hour_b23 <- 'Failed'
                    testlik_hour <- list(pv='Failed')
                }
                else {
                    logLik_intercept_hour <- logLik(model_hour)
                    coefficients_intercept_hour_b0 <- t(coef(model_hour))[1,][1]
                    coefficients_intercept_hour_b1 <- t(coef(model_hour))[1,][2]
                    coefficients_intercept_hour_b2 <- t(coef(model_hour))[1,][3]
                    coefficients_intercept_hour_b3 <- t(coef(model_hour))[1,][4]
                    coefficients_intercept_hour_b4 <- t(coef(model_hour))[1,][5]
                    coefficients_intercept_hour_b5 <- t(coef(model_hour))[1,][6]
                    coefficients_intercept_hour_b6 <- t(coef(model_hour))[1,][7]
                    coefficients_intercept_hour_b7 <- t(coef(model_hour))[1,][8]
                    coefficients_intercept_hour_b8 <- t(coef(model_hour))[1,][9]
                    coefficients_intercept_hour_b9 <- t(coef(model_hour))[1,][10]
                    coefficients_intercept_hour_b10 <- t(coef(model_hour))[1,][11]
                    coefficients_intercept_hour_b11 <- t(coef(model_hour))[1,][12]
                    coefficients_intercept_hour_b12 <- t(coef(model_hour))[1,][13]
                    coefficients_intercept_hour_b13 <- t(coef(model_hour))[1,][14]
                    coefficients_intercept_hour_b14 <- t(coef(model_hour))[1,][15]
                    coefficients_intercept_hour_b15 <- t(coef(model_hour))[1,][16]
                    coefficients_intercept_hour_b16 <- t(coef(model_hour))[1,][17]
                    coefficients_intercept_hour_b17 <- t(coef(model_hour))[1,][18]
                    coefficients_intercept_hour_b18 <- t(coef(model_hour))[1,][19]
                    coefficients_intercept_hour_b19 <- t(coef(model_hour))[1,][20]
                    coefficients_intercept_hour_b20 <- t(coef(model_hour))[1,][21]
                    coefficients_intercept_hour_b21 <- t(coef(model_hour))[1,][22]
                    coefficients_intercept_hour_b22 <- t(coef(model_hour))[1,][23]
                    coefficients_intercept_hour_b23 <- t(coef(model_hour))[1,][24]
                    testlik_hour <- testlik.fun(model_hour, model_homog)
                }
                
            }
            df_model_parameters[nrow(df_model_parameters) + 1,] = c(fold,ID,logLik_intercept_day, coefficients_intercept_day_b0, coefficients_intercept_day_b1, coefficients_intercept_day_b2, coefficients_intercept_day_b3, coefficients_intercept_day_b4, coefficients_intercept_day_b5, coefficients_intercept_day_b6, logLik_intercept_hour, coefficients_intercept_hour_b0, coefficients_intercept_hour_b1, coefficients_intercept_hour_b2, coefficients_intercept_hour_b3, coefficients_intercept_hour_b4, coefficients_intercept_hour_b5, coefficients_intercept_hour_b6, coefficients_intercept_hour_b7, coefficients_intercept_hour_b8, coefficients_intercept_hour_b9, coefficients_intercept_hour_b10, coefficients_intercept_hour_b11, coefficients_intercept_hour_b12, coefficients_intercept_hour_b13, coefficients_intercept_hour_b14, coefficients_intercept_hour_b15, coefficients_intercept_hour_b16, coefficients_intercept_hour_b17, coefficients_intercept_hour_b18, coefficients_intercept_hour_b19, coefficients_intercept_hour_b20, coefficients_intercept_hour_b21, coefficients_intercept_hour_b22, coefficients_intercept_hour_b23)
            df_likelihood_comparison[nrow(df_model_parameters) + 1,] = c(fold,ID,testlik_day$pv,testlik_hour$pv)        
                
        }
        directory = "/directory/"
        write.csv(df_model_parameters, paste(directory, "df_model_parameters_onehot_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
        write.csv(df_likelihood_comparison, paste(directory, "df_likelihood_comparison_onehot_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
    }
    return(list(df_model_parameters, df_likelihood_comparison))
}
                                               
# SIN COS TRANSFORM
model_training_sin_cos <- function(channel_name){
    channel <- channel_name # 'Phone Usage'  # 'Communication App'  # 'Social media App'
    for (fold in 1:10) {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept_day'=double(), 'coefficients_intercept_day_b0'=double(), 'coefficients_intercept_day_b1'=double(), 'coefficients_intercept_day_b2'=double(), 'logLik_intercept_hour'=double(), 'coefficients_intercept_hour_b0'=double(), 'coefficients_intercept_hour_b1'=double(), 'coefficients_intercept_hour_b2'=double()) 
        df_likelihood_comparison <- data.frame('Fold'=character(), 'ID'=character(), 'Day_likelihood'=double(), 'Hour_likelihood'=double())
        for (ID in BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]
            df_current <- df_current[df_current$Mask != fold, ]
            timestamps <- which(df_current[channel] == 2)
            if (length(timestamps)==0){
                # With day
                logLik_intercept_day <- NA
                coefficients_intercept_day_b0 <- NA
                coefficients_intercept_day_b1 <- NA
                coefficients_intercept_day_b2 <- NA
                testlik_day <- list(pv=NA)
                
                # With hour
                logLik_intercept_hour <- NA
                coefficients_intercept_hour_b0 <- NA
                coefficients_intercept_hour_b1 <- NA
                coefficients_intercept_hour_b2 <- NA
                testlik_hour <- list(pv=NA)               
            } else {
                model_homog <- fitPP.fun(covariates=NULL, posE=timestamps, nobs=length(df_current$weekday), start=list(b0=0))
                logLik_intercept_day <- 'Failed' 
                coefficients_intercept_day_b0 <- 'Failed'
                coefficients_intercept_day_b1 <- 'Failed'
                coefficients_intercept_day_b2 <- 'Failed'
                testlik_day <- list(pv='Failed')
    
                model_hour <- fitPP.fun(covariates=cbind(df_current$sin_hour,df_current$cos_hour), posE=timestamps, nobs=length(df_current$hour), start=list(b0=0,b1=0,b2=0))
                logLik_intercept_hour <- logLik(model_hour)
                coefficients_intercept_hour_b0 <- t(coef(model_hour))[1,][1]
                coefficients_intercept_hour_b1 <- t(coef(model_hour))[1,][2]
                coefficients_intercept_hour_b2 <- t(coef(model_hour))[1,][3]
                testlik_hour <- testlik.fun(model_hour, model_homog)                
            }
            df_model_parameters[nrow(df_model_parameters) + 1,] = c(fold,ID,logLik_intercept_day,coefficients_intercept_day_b0,coefficients_intercept_day_b1,coefficients_intercept_day_b2,logLik_intercept_hour,coefficients_intercept_hour_b0,coefficients_intercept_hour_b1,coefficients_intercept_hour_b2)
            df_likelihood_comparison[nrow(df_model_parameters) + 1,] = c(fold,ID,testlik_day$pv,testlik_hour$pv)        
        }
        directory = "/directory/"
        write.csv(df_model_parameters, paste(directory, "df_model_parameters_sincos_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
        write.csv(df_likelihood_comparison, paste(directory, "df_likelihood_comparison_sincos_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
    }
    return(list(df_model_parameters, df_likelihood_comparison))
}
         
# ONE HOT TRANSFORM - HOUR AND DAY COMBINED
model_training_one_hot_hour_day_combined <- function(channel_name,subset_BEHAPP_ID_list){
    channel <- channel_name # 'Phone Usage'  # 'Communication App'  # 'Social media App'
    for (fold in 1:10) {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), logLik_intercept_day_hour=double(), coefficients_intercept_day_hour_b0=double(), coefficients_intercept_day_hour_b1=double(), coefficients_intercept_day_hour_b2=double(), coefficients_intercept_day_hour_b3=double(), coefficients_intercept_day_hour_b4=double(), coefficients_intercept_day_hour_b5=double(), coefficients_intercept_day_hour_b6=double(), coefficients_intercept_day_hour_b7=double(), coefficients_intercept_day_hour_b8=double(), coefficients_intercept_day_hour_b9=double(), coefficients_intercept_day_hour_b10=double(), coefficients_intercept_day_hour_b11=double(), coefficients_intercept_day_hour_b12=double(), coefficients_intercept_day_hour_b13=double(), coefficients_intercept_day_hour_b14=double(), coefficients_intercept_day_hour_b15=double(), coefficients_intercept_day_hour_b16=double(), coefficients_intercept_day_hour_b17=double(), coefficients_intercept_day_hour_b18=double(), coefficients_intercept_day_hour_b19=double(), coefficients_intercept_day_hour_b20=double(), coefficients_intercept_day_hour_b21=double(), coefficients_intercept_day_hour_b22=double(), coefficients_intercept_day_hour_b23=double(), coefficients_intercept_day_hour_b24=double(), coefficients_intercept_day_hour_b25=double(), coefficients_intercept_day_hour_b26=double(), coefficients_intercept_day_hour_b27=double(), coefficients_intercept_day_hour_b28=double(), coefficients_intercept_day_hour_b29=double()) 
        df_likelihood_comparison <- data.frame('Fold'=character(), 'ID'=character(), 'Day_hour_likelihood'=double())
        for (ID in subset_BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]
            df_current <- df_current[df_current$Mask != fold, ]
            timestamps <- which(df_current[channel] == 2)
            if (length(timestamps)==0){               
                # With day and hour
                logLik_intercept_day_hour <- NA
                
            } else {
                # Hour model
                model_hour <- try(fitPP.fun(covariates=cbind(df_current$onehot_hour_1,df_current$onehot_hour_2,df_current$onehot_hour_3,df_current$onehot_hour_4,df_current$onehot_hour_5,df_current$onehot_hour_6,df_current$onehot_hour_7,df_current$onehot_hour_8,df_current$onehot_hour_9,df_current$onehot_hour_10,df_current$onehot_hour_11,df_current$onehot_hour_12,df_current$onehot_hour_13,df_current$onehot_hour_14,df_current$onehot_hour_15,df_current$onehot_hour_16,df_current$onehot_hour_17,df_current$onehot_hour_18,df_current$onehot_hour_19,df_current$onehot_hour_20,df_current$onehot_hour_21,df_current$onehot_hour_22,df_current$onehot_hour_23), posE=timestamps, nobs=length(df_current$hour), start=list(b0=0,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0,b7=0,b8=0,b9=0,b10=0,b11=0,b12=0,b13=0,b14=0,b15=0,b16=0,b17=0,b18=0,b19=0,b20=0,b21=0,b22=0,b23=0),lambdaylim=c(-1000,1000)))
                if (inherits(model_hour, 'try-error')){
                    logLik_intercept_hour <- 'Failed'
                    coefficients_intercept_hour_b0 <- 'Failed'
                    coefficients_intercept_hour_b1 <- 'Failed'
                    coefficients_intercept_hour_b2 <- 'Failed'
                    coefficients_intercept_hour_b3 <- 'Failed'
                    coefficients_intercept_hour_b4 <- 'Failed'
                    coefficients_intercept_hour_b5 <- 'Failed'
                    coefficients_intercept_hour_b6 <- 'Failed'
                    coefficients_intercept_hour_b7 <- 'Failed'
                    coefficients_intercept_hour_b8 <- 'Failed'
                    coefficients_intercept_hour_b9 <- 'Failed'
                    coefficients_intercept_hour_b10 <- 'Failed'
                    coefficients_intercept_hour_b11 <- 'Failed'
                    coefficients_intercept_hour_b12 <- 'Failed'
                    coefficients_intercept_hour_b13 <- 'Failed'
                    coefficients_intercept_hour_b14 <- 'Failed'
                    coefficients_intercept_hour_b15 <- 'Failed'
                    coefficients_intercept_hour_b16 <- 'Failed'
                    coefficients_intercept_hour_b17 <- 'Failed'
                    coefficients_intercept_hour_b18 <- 'Failed'
                    coefficients_intercept_hour_b19 <- 'Failed'
                    coefficients_intercept_hour_b20 <- 'Failed'
                    coefficients_intercept_hour_b21 <- 'Failed'
                    coefficients_intercept_hour_b22 <- 'Failed'
                    coefficients_intercept_hour_b23 <- 'Failed'
                    testlik_hour <- list(pv='Failed')

                    logLik_intercept_day_hour <- 'Failed'
                    coefficients_intercept_day_hour_b0 <- 'Failed'
                    coefficients_intercept_day_hour_b1 <- 'Failed'
                    coefficients_intercept_day_hour_b2 <- 'Failed'
                    coefficients_intercept_day_hour_b3 <- 'Failed'
                    coefficients_intercept_day_hour_b4 <- 'Failed'
                    coefficients_intercept_day_hour_b5 <- 'Failed'
                    coefficients_intercept_day_hour_b6 <- 'Failed'
                    coefficients_intercept_day_hour_b7 <- 'Failed'
                    coefficients_intercept_day_hour_b8 <- 'Failed'
                    coefficients_intercept_day_hour_b9 <- 'Failed'
                    coefficients_intercept_day_hour_b10 <- 'Failed'
                    coefficients_intercept_day_hour_b11 <- 'Failed'
                    coefficients_intercept_day_hour_b12 <- 'Failed'
                    coefficients_intercept_day_hour_b13 <- 'Failed'
                    coefficients_intercept_day_hour_b14 <- 'Failed'
                    coefficients_intercept_day_hour_b15 <- 'Failed'
                    coefficients_intercept_day_hour_b16 <- 'Failed'
                    coefficients_intercept_day_hour_b17 <- 'Failed'
                    coefficients_intercept_day_hour_b18 <- 'Failed'
                    coefficients_intercept_day_hour_b19 <- 'Failed'
                    coefficients_intercept_day_hour_b20 <- 'Failed'
                    coefficients_intercept_day_hour_b21 <- 'Failed'
                    coefficients_intercept_day_hour_b22 <- 'Failed'
                    coefficients_intercept_day_hour_b23 <- 'Failed'
                    coefficients_intercept_day_hour_b24 <- 'Failed'
                    coefficients_intercept_day_hour_b25 <- 'Failed'
                    coefficients_intercept_day_hour_b26 <- 'Failed'
                    coefficients_intercept_day_hour_b27 <- 'Failed'
                    coefficients_intercept_day_hour_b28 <- 'Failed'
                    coefficients_intercept_day_hour_b29 <- 'Failed'
                    testlik_day_hour <- list(pv='Failed')
                }
                else {
                    logLik_intercept_hour <- logLik(model_hour)
                    coefficients_intercept_hour_b0 <- t(coef(model_hour))[1,][1]
                    coefficients_intercept_hour_b1 <- t(coef(model_hour))[1,][2]
                    coefficients_intercept_hour_b2 <- t(coef(model_hour))[1,][3]
                    coefficients_intercept_hour_b3 <- t(coef(model_hour))[1,][4]
                    coefficients_intercept_hour_b4 <- t(coef(model_hour))[1,][5]
                    coefficients_intercept_hour_b5 <- t(coef(model_hour))[1,][6]
                    coefficients_intercept_hour_b6 <- t(coef(model_hour))[1,][7]
                    coefficients_intercept_hour_b7 <- t(coef(model_hour))[1,][8]
                    coefficients_intercept_hour_b8 <- t(coef(model_hour))[1,][9]
                    coefficients_intercept_hour_b9 <- t(coef(model_hour))[1,][10]
                    coefficients_intercept_hour_b10 <- t(coef(model_hour))[1,][11]
                    coefficients_intercept_hour_b11 <- t(coef(model_hour))[1,][12]
                    coefficients_intercept_hour_b12 <- t(coef(model_hour))[1,][13]
                    coefficients_intercept_hour_b13 <- t(coef(model_hour))[1,][14]
                    coefficients_intercept_hour_b14 <- t(coef(model_hour))[1,][15]
                    coefficients_intercept_hour_b15 <- t(coef(model_hour))[1,][16]
                    coefficients_intercept_hour_b16 <- t(coef(model_hour))[1,][17]
                    coefficients_intercept_hour_b17 <- t(coef(model_hour))[1,][18]
                    coefficients_intercept_hour_b18 <- t(coef(model_hour))[1,][19]
                    coefficients_intercept_hour_b19 <- t(coef(model_hour))[1,][20]
                    coefficients_intercept_hour_b20 <- t(coef(model_hour))[1,][21]
                    coefficients_intercept_hour_b21 <- t(coef(model_hour))[1,][22]
                    coefficients_intercept_hour_b22 <- t(coef(model_hour))[1,][23]
                    coefficients_intercept_hour_b23 <- t(coef(model_hour))[1,][24]
                           
                    # Day + hour model
                    model_day_hour <- try(fitPP.fun(covariates=cbind(df_current$onehot_weekday_1,df_current$onehot_weekday_2,df_current$onehot_weekday_3,df_current$onehot_weekday_4,df_current$onehot_weekday_5,df_current$onehot_weekday_6,df_current$onehot_hour_1,df_current$onehot_hour_2,df_current$onehot_hour_3,df_current$onehot_hour_4,df_current$onehot_hour_5,df_current$onehot_hour_6,df_current$onehot_hour_7,df_current$onehot_hour_8,df_current$onehot_hour_9,df_current$onehot_hour_10,df_current$onehot_hour_11,df_current$onehot_hour_12,df_current$onehot_hour_13,df_current$onehot_hour_14,df_current$onehot_hour_15,df_current$onehot_hour_16,df_current$onehot_hour_17,df_current$onehot_hour_18,df_current$onehot_hour_19,df_current$onehot_hour_20,df_current$onehot_hour_21,df_current$onehot_hour_22,df_current$onehot_hour_23), posE=timestamps, nobs=length(df_current$hour), start=list(b0=0,b1=0,b2=0,b3=0,b4=0,b5=0,b6=0,b7=0,b8=0,b9=0,b10=0,b11=0,b12=0,b13=0,b14=0,b15=0,b16=0,b17=0,b18=0,b19=0,b20=0,b21=0,b22=0,b23=0,b24=0,b25=0,b26=0,b27=0,b28=0,b29=0),lambdaylim=c(-1000,1000)))
                    if (inherits(model_day_hour, 'try-error')){
                        logLik_intercept_day_hour <- 'Failed'
                        coefficients_intercept_day_hour_b0 <- 'Failed'
                        coefficients_intercept_day_hour_b1 <- 'Failed'
                        coefficients_intercept_day_hour_b2 <- 'Failed'
                        coefficients_intercept_day_hour_b3 <- 'Failed'
                        coefficients_intercept_day_hour_b4 <- 'Failed'
                        coefficients_intercept_day_hour_b5 <- 'Failed'
                        coefficients_intercept_day_hour_b6 <- 'Failed'
                        coefficients_intercept_day_hour_b7 <- 'Failed'
                        coefficients_intercept_day_hour_b8 <- 'Failed'
                        coefficients_intercept_day_hour_b9 <- 'Failed'
                        coefficients_intercept_day_hour_b10 <- 'Failed'
                        coefficients_intercept_day_hour_b11 <- 'Failed'
                        coefficients_intercept_day_hour_b12 <- 'Failed'
                        coefficients_intercept_day_hour_b13 <- 'Failed'
                        coefficients_intercept_day_hour_b14 <- 'Failed'
                        coefficients_intercept_day_hour_b15 <- 'Failed'
                        coefficients_intercept_day_hour_b16 <- 'Failed'
                        coefficients_intercept_day_hour_b17 <- 'Failed'
                        coefficients_intercept_day_hour_b18 <- 'Failed'
                        coefficients_intercept_day_hour_b19 <- 'Failed'
                        coefficients_intercept_day_hour_b20 <- 'Failed'
                        coefficients_intercept_day_hour_b21 <- 'Failed'
                        coefficients_intercept_day_hour_b22 <- 'Failed'
                        coefficients_intercept_day_hour_b23 <- 'Failed'
                        coefficients_intercept_day_hour_b24 <- 'Failed'
                        coefficients_intercept_day_hour_b25 <- 'Failed'
                        coefficients_intercept_day_hour_b26 <- 'Failed'
                        coefficients_intercept_day_hour_b27 <- 'Failed'
                        coefficients_intercept_day_hour_b28 <- 'Failed'
                        coefficients_intercept_day_hour_b29 <- 'Failed'
                        testlik_day_hour <- list(pv='Failed')
                    }
                    else {
                        logLik_intercept_day_hour <- logLik(model_day_hour)
                        coefficients_intercept_day_hour_b0 <- t(coef(model_day_hour))[1,][1]
                        coefficients_intercept_day_hour_b1 <- t(coef(model_day_hour))[1,][2]
                        coefficients_intercept_day_hour_b2 <- t(coef(model_day_hour))[1,][3]
                        coefficients_intercept_day_hour_b3 <- t(coef(model_day_hour))[1,][4]
                        coefficients_intercept_day_hour_b4 <- t(coef(model_day_hour))[1,][5]
                        coefficients_intercept_day_hour_b5 <- t(coef(model_day_hour))[1,][6]
                        coefficients_intercept_day_hour_b6 <- t(coef(model_day_hour))[1,][7]
                        coefficients_intercept_day_hour_b7 <- t(coef(model_day_hour))[1,][8]
                        coefficients_intercept_day_hour_b8 <- t(coef(model_day_hour))[1,][9]
                        coefficients_intercept_day_hour_b9 <- t(coef(model_day_hour))[1,][10]
                        coefficients_intercept_day_hour_b10 <- t(coef(model_day_hour))[1,][11]
                        coefficients_intercept_day_hour_b11 <- t(coef(model_day_hour))[1,][12]
                        coefficients_intercept_day_hour_b12 <- t(coef(model_day_hour))[1,][13]
                        coefficients_intercept_day_hour_b13 <- t(coef(model_day_hour))[1,][14]
                        coefficients_intercept_day_hour_b14 <- t(coef(model_day_hour))[1,][15]
                        coefficients_intercept_day_hour_b15 <- t(coef(model_day_hour))[1,][16]
                        coefficients_intercept_day_hour_b16 <- t(coef(model_day_hour))[1,][17]
                        coefficients_intercept_day_hour_b17 <- t(coef(model_day_hour))[1,][18]
                        coefficients_intercept_day_hour_b18 <- t(coef(model_day_hour))[1,][19]
                        coefficients_intercept_day_hour_b19 <- t(coef(model_day_hour))[1,][20]
                        coefficients_intercept_day_hour_b20 <- t(coef(model_day_hour))[1,][21]
                        coefficients_intercept_day_hour_b21 <- t(coef(model_day_hour))[1,][22]
                        coefficients_intercept_day_hour_b22 <- t(coef(model_day_hour))[1,][23]
                        coefficients_intercept_day_hour_b23 <- t(coef(model_day_hour))[1,][24]
                        coefficients_intercept_day_hour_b24 <- t(coef(model_day_hour))[1,][25]
                        coefficients_intercept_day_hour_b25 <- t(coef(model_day_hour))[1,][26]
                        coefficients_intercept_day_hour_b26 <- t(coef(model_day_hour))[1,][27]
                        coefficients_intercept_day_hour_b27 <- t(coef(model_day_hour))[1,][28]
                        coefficients_intercept_day_hour_b28 <- t(coef(model_day_hour))[1,][29]
                        coefficients_intercept_day_hour_b29 <- t(coef(model_day_hour))[1,][30]
                        testlik_day_hour <- testlik.fun(model_day_hour, model_hour) 
                    }                 
                }
            }
            df_model_parameters[nrow(df_model_parameters) + 1,] = c(fold,ID,logLik_intercept_day_hour, coefficients_intercept_day_hour_b0, coefficients_intercept_day_hour_b1, coefficients_intercept_day_hour_b2, coefficients_intercept_day_hour_b3, coefficients_intercept_day_hour_b4, coefficients_intercept_day_hour_b5, coefficients_intercept_day_hour_b6, coefficients_intercept_day_hour_b7, coefficients_intercept_day_hour_b8, coefficients_intercept_day_hour_b9, coefficients_intercept_day_hour_b10, coefficients_intercept_day_hour_b11, coefficients_intercept_day_hour_b12, coefficients_intercept_day_hour_b13, coefficients_intercept_day_hour_b14, coefficients_intercept_day_hour_b15, coefficients_intercept_day_hour_b16, coefficients_intercept_day_hour_b17, coefficients_intercept_day_hour_b18, coefficients_intercept_day_hour_b19, coefficients_intercept_day_hour_b20, coefficients_intercept_day_hour_b21, coefficients_intercept_day_hour_b22, coefficients_intercept_day_hour_b23, coefficients_intercept_day_hour_b24, coefficients_intercept_day_hour_b25, coefficients_intercept_day_hour_b26, coefficients_intercept_day_hour_b27, coefficients_intercept_day_hour_b28, coefficients_intercept_day_hour_b29)
            df_likelihood_comparison[nrow(df_model_parameters) + 1,] = c(fold,ID,testlik_day_hour$pv)        
        }
        directory = "/directory/"
        write.csv(df_model_parameters, paste(directory, "df_model_parameters_day-hour-subset_onehot_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
        write.csv(df_likelihood_comparison, paste(directory, "df_likelihood_comparison_day-hour-to-hour_onehot_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
    }
    return(list(df_model_parameters, df_likelihood_comparison))
}

# SIN COS TRANSFORM - HOUR AND DAY COMBINED
model_training_sin_cos_hour_day_combined <- function(channel_name,subset_BEHAPP_ID_list){
    channel <- channel_name # 'Phone Usage'  # 'Communication App'  # 'Social media App'
    for (fold in 1:10) {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept_day_hour'=double(), 'coefficients_intercept_day_hour_b0'=double(), 'coefficients_intercept_day_hour_b1'=double(), 'coefficients_intercept_day_hour_b2'=double(), 'coefficients_intercept_day_hour_b3'=double(), 'coefficients_intercept_day_hour_b4'=double()) 
        df_likelihood_comparison <- data.frame('Fold'=character(), 'ID'=character(), 'Day_hour_likelihood'=double())
        for (ID in subset_BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]
            df_current <- df_current[df_current$Mask != fold, ]
            timestamps <- which(df_current[channel] == 2)
            if (length(timestamps)==0){               
                # With hour
                logLik_intercept_hour <- NA
                coefficients_intercept_hour_b0 <- NA
                coefficients_intercept_hour_b1 <- NA
                coefficients_intercept_hour_b2 <- NA
                testlik_hour <- list(pv=NA)
                
                # With day and hour
                logLik_intercept_day_hour <- NA
                coefficients_intercept_day_hour_b0 <- NA
                coefficients_intercept_day_hour_b1 <- NA
                coefficients_intercept_day_hour_b2 <- NA
                coefficients_intercept_day_hour_b3 <- NA
                coefficients_intercept_day_hour_b4 <- NA
                testlik_day_hour <- list(pv=NA)
                
            } else {
                model_hour <- fitPP.fun(covariates=cbind(df_current$sin_hour,df_current$cos_hour), posE=timestamps, nobs=length(df_current$hour), start=list(b0=0,b1=0,b2=0))
                logLik_intercept_hour <- logLik(model_hour)
                coefficients_intercept_hour_b0 <- t(coef(model_hour))[1,][1]
                coefficients_intercept_hour_b1 <- t(coef(model_hour))[1,][2]
                coefficients_intercept_hour_b2 <- t(coef(model_hour))[1,][3]                
          
                model_day_hour <- fitPP.fun(covariates=cbind(df_current$sin_weekday,df_current$cos_weekday,df_current$sin_hour,df_current$cos_hour), posE=timestamps, nobs=length(df_current$sin_weekday), start=list(b0=0,b1=0,b2=0,b3=0,b4=0))
                logLik_intercept_day_hour <- logLik(model_day_hour)
                coefficients_intercept_day_hour_b0 <- t(coef(model_day_hour))[1,][1]
                coefficients_intercept_day_hour_b1 <- t(coef(model_day_hour))[1,][2]
                coefficients_intercept_day_hour_b2 <- t(coef(model_day_hour))[1,][3]
                coefficients_intercept_day_hour_b3 <- t(coef(model_day_hour))[1,][4]
                coefficients_intercept_day_hour_b4 <- t(coef(model_day_hour))[1,][5]
                testlik_day_hour <- testlik.fun(model_day_hour, model_hour) 
            }
            df_model_parameters[nrow(df_model_parameters) + 1,] = c(fold,ID,logLik_intercept_day_hour,coefficients_intercept_day_hour_b0,coefficients_intercept_day_hour_b1,coefficients_intercept_day_hour_b2,coefficients_intercept_day_hour_b3,coefficients_intercept_day_hour_b4)
            df_likelihood_comparison[nrow(df_model_parameters) + 1,] = c(fold,ID,testlik_day_hour$pv)        
        }
        directory = "/directory/"
        write.csv(df_model_parameters, paste(directory, "df_model_parameters_day-hour-subset_sincos_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
        write.csv(df_likelihood_comparison, paste(directory, "df_likelihood_comparison_day-hour-to-hour_sincos_transform_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
    }
    return(list(df_model_parameters, df_likelihood_comparison))
}


# HOMOGENEOUS POISSON MODEL
model_training_homogeneous <- function(channel_name){
    channel <- channel_name # 'Phone Usage'  # 'Communication App'  # 'Social media App'
    for (fold in 1:10) {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept'=double(), 'coefficients_intercept_b0'=double()) 
        for (ID in BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]
            df_current <- df_current[df_current$Mask != fold, ]
            timestamps <- which(df_current[channel] == 2)
            if (length(timestamps)==0){
                logLik_intercept <- NA
                coefficients_intercept_b0 <- NA                
            } else {
                model_homog <- fitPP.fun(covariates=NULL, posE=timestamps, nobs=length(df_current$weekday), start=list(b0=0))
                logLik_intercept <- logLik(model_homog)
                coefficients_intercept_b0 <- t(coef(model_homog))[1,][1]                
            }
            df_model_parameters[nrow(df_model_parameters) + 1,] = c(fold,ID,logLik_intercept,coefficients_intercept_b0)
        }
        directory = "/directory/"
        write.csv(df_model_parameters, paste(directory, "df_model_parameters_homogeneous_fold", toString(fold), "_", channel,".csv", sep=""), row.names = FALSE)
    }
    return(df_model_parameters)
}

read_saved_models <- function(folder, channel, transform) {
    # Read csv's back in for df_model_parameters as have separate files for each fold:
    df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept_day'=double(), 'coefficients_intercept_day_b0'=double(), 'coefficients_intercept_day_b1'=double(), 'logLik_intercept_day_percent'=double(), 'coefficients_intercept_day_percent_b0'=double(), 'coefficients_intercept_day_percent_b1'=double(), 'logLik_intercept_day_hour'=double(), 'coefficients_intercept_day_hour_b0'=double(), 'coefficients_intercept_day_hour_b1'=double(), 'coefficients_intercept_day_hour_b2'=double(), 'logLik_intercept_day_hour_percent'=double(), 'coefficients_intercept_day_hour_percent_b0'=double(), 'coefficients_intercept_day_hour_percent_b1'=double(), 'coefficients_intercept_day_hour_percent_b2'=double()) 
    if (transform == "_onehot_transform") {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), logLik_intercept_day=double(), coefficients_intercept_day_b0=double(), coefficients_intercept_day_b1=double(), coefficients_intercept_day_b2=double(), coefficients_intercept_day_b3=double(), coefficients_intercept_day_b4=double(), coefficients_intercept_day_b5=double(), coefficients_intercept_day_b6=double(), logLik_intercept_hour=double(), coefficients_intercept_hour_b0=double(), coefficients_intercept_hour_b1=double(), coefficients_intercept_hour_b2=double(), coefficients_intercept_hour_b3=double(), coefficients_intercept_hour_b4=double(), coefficients_intercept_hour_b5=double(), coefficients_intercept_hour_b6=double(), coefficients_intercept_hour_b7=double(), coefficients_intercept_hour_b8=double(), coefficients_intercept_hour_b9=double(), coefficients_intercept_hour_b10=double(), coefficients_intercept_hour_b11=double(), coefficients_intercept_hour_b12=double(), coefficients_intercept_hour_b13=double(), coefficients_intercept_hour_b14=double(), coefficients_intercept_hour_b15=double(), coefficients_intercept_hour_b16=double(), coefficients_intercept_hour_b17=double(), coefficients_intercept_hour_b18=double(), coefficients_intercept_hour_b19=double(), coefficients_intercept_hour_b20=double(), coefficients_intercept_hour_b21=double(), coefficients_intercept_hour_b22=double(), coefficients_intercept_hour_b23=double()) 
    }
    if (transform == "_sincos_transform") {
        df_model_parameters <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept_day'=double(), 'coefficients_intercept_day_b0'=double(), 'coefficients_intercept_day_b1'=double(), 'coefficients_intercept_day_b2'=double(), 'logLik_intercept_hour'=double(), 'coefficients_intercept_hour_b0'=double(), 'coefficients_intercept_hour_b1'=double(), 'coefficients_intercept_hour_b2'=double()) 
    }
    if (transform == "_homogeneous") {
        df_model_parameters_homog <- data.frame('Fold'=character(), 'ID'=character(), 'logLik_intercept'=double(), 'coefficients_intercept_b0'=double()) 
    }
    for (fold in 1:10) {
        file = paste("/directory/",folder,"/",channel,"/Model Parameters/df_model_parameters",transform,"_fold",toString(fold),"_",channel,".csv", sep="")   
        df_fold <- read.csv(file)
        df_model_parameters <- rbind(df_model_parameters, df_fold)
    }
    return(df_model_parameters)
}

read_saved_lrts <- function(folder, channel, transform) {
    # Read csv's back in for df_likelihood_comparison as have separate files for each fold:
    df_lrt <- data.frame('Fold'=character(), 'ID'=character(), 'Day_likelihood'=double(), 'Hour_likelihood'=double()) 
    for (fold in 1:10) {
        file = paste("/directory/",folder,"/",channel,"/Likelihood Comparison/df_likelihood_comparison",transform,"_fold",toString(fold),"_",channel,".csv", sep="")   
        df_fold <- read.csv(file)
        df_lrt <- rbind(df_lrt, df_fold)
    }
    return(df_lrt)
}

In [ ]:
%%R
# Simulation of points
# Baseline models
simulate_points_baselines <- function(channel, model_type, dataframe, IDs) {  
    BEHAPP_ID_list = IDs
    model = model_type  #model_homog
    simulated_timestamps_count = list()
    simulated_timestamps_series = list()
    out_of_sample_likelihood = list()
    for (fold in 1:10) {
        for (ID in BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]  # df with masking
            ID_parameters <- df_model_parameters[df_model_parameters$ID == ID,]
            basic_day_cycle = 0:6
            basic_hour_cycle = 0:23
            if (is.na(ID_parameters$coefficients_intercept_b0[fold]) == TRUE) {
                count = NA
                out_of_sample_likelihood <- append(out_of_sample_likelihood,NA)
            } else {               
                # Find length of masked section
                df_current <- df_current[df_current$Mask == fold, ]
                masked_length <- nrow(df_current)
                if (model == "model_homog") {
                    X.time.span <- c(rep(1,masked_length))
                    beta <- as.numeric(ID_parameters$coefficients_intercept_b0[fold])
                    lambda <- exp(X.time.span * beta) # Calculate lambda for each of the timepoints
                    # Calculate out of sample likelihood
                    # Activity points
                    df_X <- as.data.frame(X.time.span)
                    df_X$channel <- df_current[channel]
                    df_points_with_activity <- subset(df_X, channel==2) # Only keep the rows with activity points
                    df_points_with_activity <- subset(df_points_with_activity, select=-channel)
                    X.points_with_activity <- as.matrix(df_points_with_activity)
                    likelihood <- -sum(lambda) + sum(X.points_with_activity %*% beta) # Equation from section 14.2 in NHPoisson
                    out_of_sample_likelihood <- c(out_of_sample_likelihood,likelihood)
                    simulated_timestamps <- simNHP.fun(lambda=lambda, fixed.seed=3)
                    count = length(simulated_timestamps$posNH)
                    simulated_timestamps_indices <- list(simulated_timestamps$posNH)
                }         
            }
            simulated_timestamps_count <- c(simulated_timestamps_count, count)    # Add indices of events to list
            simulated_timestamps_series <- c(simulated_timestamps_series, simulated_timestamps_indices)
        }
    }
    fold_list = df_model_parameters$Fold
    ID_list = df_model_parameters$ID 
    return(list(out_of_sample_likelihood=out_of_sample_likelihood, simulated_timestamps_count=simulated_timestamps_count, simulated_timestamps_series=simulated_timestamps_series, fold_list=fold_list, ID_list=ID_list))
}

simulate_points <- function(channel, model_type, dataframe, IDs, basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle){  
    model = model_type  # model_hour model_day model_day_hour
    df_ID_app = dataframe
    BEHAPP_ID_list = IDs
    basic_sin_day_cycle = array(unlist(basic_sin_day_cycle))
    basic_cos_day_cycle = array(unlist(basic_cos_day_cycle))
    basic_sin_hour_cycle = array(unlist(basic_sin_hour_cycle))
    basic_cos_hour_cycle = array(unlist(basic_cos_hour_cycle))
    simulated_timestamps_count = list()
    simulated_timestamps_series = list()
    out_of_sample_likelihood = list()
    for (fold in 1:10) {
        for (ID in BEHAPP_ID_list) {
            df_current <- df_ID_app[df_ID_app$ID == ID,]  # df with masking
            ID_parameters <- df_model_parameters[df_model_parameters$ID == ID,]
            basic_day_cycle = 0:6
            basic_hour_cycle = 0:23
            if (is.na(ID_parameters[ID_parameters$Fold==fold,3]) == TRUE) {
                count = NA
                out_of_sample_likelihood <- append(out_of_sample_likelihood,NA)
            } else { 
                # Find day and hour of first masked value in time series before masking so know where to start from
                masked_start_day <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$weekday
                masked_start_day_index <- match(masked_start_day, basic_day_cycle)
                participant_specific_day_cycle <- c(basic_day_cycle[masked_start_day_index:length(basic_day_cycle)], basic_day_cycle[1:masked_start_day_index-1])
                masked_start_hour <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$hour
                masked_start_hour_index <- match(masked_start_hour, basic_hour_cycle)
                participant_specific_hour_cycle <- c(basic_hour_cycle[masked_start_hour_index:length(basic_hour_cycle)], basic_hour_cycle[1:masked_start_hour_index-1])
                if (model == "model_day_sincos" || model == "model_hour_sincos" || model == "model_day_hour_sincos") {
                    # Transformed - sin cos
                    masked_start_sin_day <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$sin_weekday
                    masked_start_sin_day_index <- match(masked_start_sin_day, basic_sin_day_cycle)
                    participant_specific_sin_day_cycle <- c(basic_sin_day_cycle[masked_start_sin_day_index:length(basic_sin_day_cycle)], basic_sin_day_cycle[1:masked_start_sin_day_index-1])
                    masked_start_cos_day <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$cos_weekday
                    masked_start_cos_day_index <- match(masked_start_cos_day, basic_cos_day_cycle)
                    participant_specific_cos_day_cycle <- c(basic_cos_day_cycle[masked_start_cos_day_index:length(basic_cos_day_cycle)], basic_cos_day_cycle[1:masked_start_cos_day_index-1])
                    
                    masked_start_sin_hour <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$sin_hour
                    masked_start_sin_hour_index <- match(masked_start_sin_hour, basic_sin_hour_cycle)
                    participant_specific_sin_hour_cycle <- c(basic_sin_hour_cycle[masked_start_sin_hour_index:length(basic_sin_hour_cycle)], basic_sin_hour_cycle[1:masked_start_sin_hour_index-1])
                    masked_start_cos_hour <- df_current[which(df_current$Mask == fold, arr.ind=TRUE)[1],]$cos_hour
                    masked_start_cos_hour_index <- match(masked_start_cos_hour, basic_cos_hour_cycle)
                    participant_specific_cos_hour_cycle <- c(basic_cos_hour_cycle[masked_start_cos_hour_index:length(basic_cos_hour_cycle)], basic_cos_hour_cycle[1:masked_start_cos_hour_index-1])
                }
                # One-hot encoded
                participant_specific_onehot_day_cycle1 <- list()
                participant_specific_onehot_day_cycle2 <- list()
                participant_specific_onehot_day_cycle3 <- list()
                participant_specific_onehot_day_cycle4 <- list()
                participant_specific_onehot_day_cycle5 <- list()
                participant_specific_onehot_day_cycle6 <- list()
                for (c in participant_specific_day_cycle){
                    participant_specific_onehot_day_cycle1 <- append(participant_specific_onehot_day_cycle1, basic_onehot_day_matrix_cycle[c+1,1])  
                    participant_specific_onehot_day_cycle2 <- append(participant_specific_onehot_day_cycle2, basic_onehot_day_matrix_cycle[c+1,2]) 
                    participant_specific_onehot_day_cycle3 <- append(participant_specific_onehot_day_cycle3, basic_onehot_day_matrix_cycle[c+1,3]) 
                    participant_specific_onehot_day_cycle4 <- append(participant_specific_onehot_day_cycle4, basic_onehot_day_matrix_cycle[c+1,4]) 
                    participant_specific_onehot_day_cycle5 <- append(participant_specific_onehot_day_cycle5, basic_onehot_day_matrix_cycle[c+1,5]) 
                    participant_specific_onehot_day_cycle6 <- append(participant_specific_onehot_day_cycle6, basic_onehot_day_matrix_cycle[c+1,6]) 
                }

                participant_specific_onehot_hour_cycle1 <- list()
                participant_specific_onehot_hour_cycle2 <- list()
                participant_specific_onehot_hour_cycle3 <- list()
                participant_specific_onehot_hour_cycle4 <- list()
                participant_specific_onehot_hour_cycle5 <- list()
                participant_specific_onehot_hour_cycle6 <- list()
                participant_specific_onehot_hour_cycle7 <- list()
                participant_specific_onehot_hour_cycle8 <- list()
                participant_specific_onehot_hour_cycle9 <- list()
                participant_specific_onehot_hour_cycle10 <- list()
                participant_specific_onehot_hour_cycle11 <- list()
                participant_specific_onehot_hour_cycle12 <- list()                
                participant_specific_onehot_hour_cycle13 <- list()
                participant_specific_onehot_hour_cycle14 <- list()
                participant_specific_onehot_hour_cycle15 <- list()
                participant_specific_onehot_hour_cycle16 <- list()
                participant_specific_onehot_hour_cycle17 <- list()
                participant_specific_onehot_hour_cycle18 <- list()
                participant_specific_onehot_hour_cycle19 <- list()
                participant_specific_onehot_hour_cycle20 <- list()
                participant_specific_onehot_hour_cycle21 <- list()
                participant_specific_onehot_hour_cycle22 <- list()
                participant_specific_onehot_hour_cycle23 <- list()
                for (c in participant_specific_hour_cycle){
                    participant_specific_onehot_hour_cycle1 <- append(participant_specific_onehot_hour_cycle1, basic_onehot_hour_matrix_cycle[c+1,1])  
                    participant_specific_onehot_hour_cycle2 <- append(participant_specific_onehot_hour_cycle2, basic_onehot_hour_matrix_cycle[c+1,2]) 
                    participant_specific_onehot_hour_cycle3 <- append(participant_specific_onehot_hour_cycle3, basic_onehot_hour_matrix_cycle[c+1,3]) 
                    participant_specific_onehot_hour_cycle4 <- append(participant_specific_onehot_hour_cycle4, basic_onehot_hour_matrix_cycle[c+1,4]) 
                    participant_specific_onehot_hour_cycle5 <- append(participant_specific_onehot_hour_cycle5, basic_onehot_hour_matrix_cycle[c+1,5]) 
                    participant_specific_onehot_hour_cycle6 <- append(participant_specific_onehot_hour_cycle6, basic_onehot_hour_matrix_cycle[c+1,6]) 
                    participant_specific_onehot_hour_cycle7 <- append(participant_specific_onehot_hour_cycle7, basic_onehot_hour_matrix_cycle[c+1,7])  
                    participant_specific_onehot_hour_cycle8 <- append(participant_specific_onehot_hour_cycle8, basic_onehot_hour_matrix_cycle[c+1,8]) 
                    participant_specific_onehot_hour_cycle9 <- append(participant_specific_onehot_hour_cycle9, basic_onehot_hour_matrix_cycle[c+1,9]) 
                    participant_specific_onehot_hour_cycle10 <- append(participant_specific_onehot_hour_cycle10, basic_onehot_hour_matrix_cycle[c+1,10]) 
                    participant_specific_onehot_hour_cycle11 <- append(participant_specific_onehot_hour_cycle11, basic_onehot_hour_matrix_cycle[c+1,11]) 
                    participant_specific_onehot_hour_cycle12 <- append(participant_specific_onehot_hour_cycle12, basic_onehot_hour_matrix_cycle[c+1,12]) 
                    participant_specific_onehot_hour_cycle13 <- append(participant_specific_onehot_hour_cycle13, basic_onehot_hour_matrix_cycle[c+1,13])  
                    participant_specific_onehot_hour_cycle14 <- append(participant_specific_onehot_hour_cycle14, basic_onehot_hour_matrix_cycle[c+1,14]) 
                    participant_specific_onehot_hour_cycle15 <- append(participant_specific_onehot_hour_cycle15, basic_onehot_hour_matrix_cycle[c+1,15]) 
                    participant_specific_onehot_hour_cycle16 <- append(participant_specific_onehot_hour_cycle16, basic_onehot_hour_matrix_cycle[c+1,16]) 
                    participant_specific_onehot_hour_cycle17 <- append(participant_specific_onehot_hour_cycle17, basic_onehot_hour_matrix_cycle[c+1,17]) 
                    participant_specific_onehot_hour_cycle18 <- append(participant_specific_onehot_hour_cycle18, basic_onehot_hour_matrix_cycle[c+1,18]) 
                    participant_specific_onehot_hour_cycle19 <- append(participant_specific_onehot_hour_cycle19, basic_onehot_hour_matrix_cycle[c+1,19])  
                    participant_specific_onehot_hour_cycle20 <- append(participant_specific_onehot_hour_cycle20, basic_onehot_hour_matrix_cycle[c+1,20]) 
                    participant_specific_onehot_hour_cycle21 <- append(participant_specific_onehot_hour_cycle21, basic_onehot_hour_matrix_cycle[c+1,21]) 
                    participant_specific_onehot_hour_cycle22 <- append(participant_specific_onehot_hour_cycle22, basic_onehot_hour_matrix_cycle[c+1,22]) 
                    participant_specific_onehot_hour_cycle23 <- append(participant_specific_onehot_hour_cycle23, basic_onehot_hour_matrix_cycle[c+1,23]) 
                }
                df_current <- df_current[df_current$Mask == fold, ]             
                # Find length of entire masked section
                masked_length <- nrow(df_current)
                if (model == "model_day_sincos") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(participant_specific_sin_day_cycle,length.out=masked_length, each=24), rep(participant_specific_cos_day_cycle,length.out=masked_length, each=24))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_day_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b2[fold]))                             
                } else if (model == "model_hour_sincos") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(participant_specific_sin_hour_cycle,length.out=masked_length), rep(participant_specific_cos_hour_cycle,length.out=masked_length))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_hour_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b2[fold]))                              
                } else if (model == "model_day_hour_sincos") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(participant_specific_sin_day_cycle,length.out=masked_length, each=24), rep(participant_specific_cos_day_cycle,length.out=masked_length, each=24), rep(participant_specific_sin_hour_cycle,length.out=masked_length), rep(participant_specific_cos_hour_cycle,length.out=masked_length))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_day_hour_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b2[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b3[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b4[fold]))                              
                } else if (model == "model_day_onehot") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(unlist(participant_specific_onehot_day_cycle1),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle2),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle3),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle4),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle5),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle6),length.out=masked_length, each=24))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_day_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b2[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b3[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b4[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b5[fold]),as.numeric(ID_parameters$coefficients_intercept_day_b6[fold]))                                     
                } else if (model == "model_hour_onehot") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(unlist(participant_specific_onehot_hour_cycle1),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle2),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle3),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle4),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle5),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle6),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle7),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle8),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle9),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle10),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle11),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle12),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle13),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle14),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle15),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle16),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle17),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle18),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle19),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle20),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle21),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle22),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle23),length.out=masked_length))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_hour_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b2[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b3[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b4[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b5[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b6[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b7[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b8[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b9[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b10[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b11[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b12[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b13[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b14[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b15[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b16[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b17[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b18[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b19[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b20[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b21[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b22[fold]),as.numeric(ID_parameters$coefficients_intercept_hour_b23[fold]))                                     
                } else if (model == "model_day_hour_onehot") {
                    # Whole fold
                    X.time.span <- cbind(rep(1,masked_length), rep(unlist(participant_specific_onehot_day_cycle1),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle2),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle3),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle4),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle5),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_day_cycle6),length.out=masked_length, each=24), rep(unlist(participant_specific_onehot_hour_cycle1),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle2),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle3),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle4),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle5),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle6),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle7),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle8),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle9),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle10),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle11),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle12),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle13),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle14),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle15),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle16),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle17),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle18),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle19),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle20),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle21),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle22),length.out=masked_length), rep(unlist(participant_specific_onehot_hour_cycle23),length.out=masked_length))
                    beta <- c(as.numeric(ID_parameters$coefficients_intercept_day_hour_b0[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b1[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b2[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b3[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b4[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b5[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b6[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b7[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b8[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b9[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b10[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b11[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b12[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b13[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b14[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b15[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b16[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b17[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b18[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b19[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b20[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b21[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b22[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b23[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b24[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b25[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b26[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b27[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b28[fold]),as.numeric(ID_parameters$coefficients_intercept_day_hour_b29[fold]))                                     
                }
                lambda <- exp(X.time.span %*% beta) # Calculate lambda for each of the timepoints in the time series
                # Calculate out of sample likelihood
                # Activity points
                df_X <- as.data.frame(X.time.span)
                df_X$channel <- df_current[channel]
                df_points_with_activity <- subset(df_X, channel==2) # Only keep the rows with activity points
                df_points_with_activity <- subset(df_points_with_activity, select=-channel)
                X.points_with_activity <- as.matrix(df_points_with_activity)
                likelihood <- -sum(lambda) + sum(X.points_with_activity %*% beta) # Equation from section 14.2 in NHPoisson
                out_of_sample_likelihood <- c(out_of_sample_likelihood,likelihood)
                # Simulate timestamps
                simulated_timestamps <- simNHP.fun(lambda=lambda, fixed.seed=3)
                count = length(simulated_timestamps$posNH)
            }
            simulated_timestamps_series <- c(simulated_timestamps_series, list(simulated_timestamps$posNH))
            simulated_timestamps_count <- c(simulated_timestamps_count, count)    # Add indices of events ($posNH) to list
        }
    }
    fold_list = df_model_parameters$Fold
    ID_list = df_model_parameters$ID
    return(list(out_of_sample_likelihood=out_of_sample_likelihood, simulated_timestamps_count=simulated_timestamps_count, simulated_timestamps_series=simulated_timestamps_series, fold_list=fold_list, ID_list=ID_list))
    
}


## Preprocessing

In [ ]:
df = mask_data_cross_validation(dataframe=df,BEHAPP_ID_list=df['ID'].unique())
df, df_original = bin_data_cross_validation(dataframe=df)

## Run models

In [ ]:
# Example for phone usage
df_ID_app = df[['ID','weekday','hour','Phone Usage','Mask']].copy()

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list
model_training_homogeneous("Phone Usage")

##### Train model where covariates encoded using one-hot encoding

In [ ]:
# One-hot encoded
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='onehot')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list

df_model_parameters_likelihood_comparison_onehot <- model_training_onehot('Phone Usage')                           

In [ ]:
%%R -i df_ID_app
# One-hot hour+day models for subset of participants where one hot day models were significant in LRTs
subset_BEHAPP_ID_list_social_media_app <- c(...)  # Create ID lists
subset_BEHAPP_ID_list_communication_app <- c(...)
subset_BEHAPP_ID_list_phone_usage <- c(...)
    
df_model_parameters_likelihood_comparison_one_hot_hour_day <- model_training_one_hot_hour_day_combined('Phone Usage',subset_BEHAPP_ID_list_phone_usage)


##### Train model where covariates encoded using sine-cosine transformation

In [ ]:
# Sin cos
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='sin_cos')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list
df_model_parameters_likelihood_comparison_sin_cos <- model_training_sin_cos('Phone Usage')                                  

In [ ]:
%%R -i df_ID_app
# Sine cosine hour+day models for subset of participants where sin cos day models were significant in LRTs
subset_BEHAPP_ID_list_social_media_app <- c(...)  # Create ID lists
subset_BEHAPP_ID_list_communication_app <- c(...)
subset_BEHAPP_ID_list_phone_usage <- c(...)
    
df_model_parameters_likelihood_comparison_sin_cos_hour_day <- model_training_sin_cos_hour_day_combined('Phone Usage',subset_BEHAPP_ID_list_phone_usage)


### Read back in saved model info

In [ ]:
%%R
df_model_parameters <- read_saved_models(folder="One Hot Hour Day", "Phone Usage", "_onehot_transform")

### Look at LRTs

In [ ]:
%%R
# For main models
df_lrt_summary <- data.frame('Channel'=character(), 'Transform'=character(), 'Number of models where day significant'=double(), 'Number of models where hour significant'=double(), 'Total number of models'=double(), 'Number of failed day models'=double(), 'Number of failed hour models'=double()) 

# For day+hour combined models
df_lrt_summary_day_hour_combined <- data.frame('Channel'=character(), 'Transform'=character(), 'Number of models where day_hour significant'=double(), 'Number of failed day_hour models'=double()) 

In [ ]:
%%R
# Phone Usage - one-hot
df_lrt <- read_saved_lrts("One Hot Hour Day", "Phone Usage", "_onehot_transform")

df_lrt <- df_lrt[complete.cases(df_lrt[ ,1]),]  # As there are some completely NA rows
df_lrt$Day_likelihood <- as.numeric(as.character(df_lrt$Day_likelihood))  # Dealing with mixed types
df_lrt$Hour_likelihood <- as.numeric(as.character(df_lrt$Hour_likelihood))

df_lrt_day_significant <- subset(df_lrt, Day_likelihood<as.numeric(.05))
df_lrt_hour_significant <- subset(df_lrt, Hour_likelihood<as.numeric(.05))
df_day_failed <- subset(df_lrt, Day_likelihood=="Failed")  # If want to check this have to remove as.numeric()
df_hour_failed <- subset(df_lrt, Hour_likelihood=="Failed")
df_lrt_summary <- rbind(df_lrt_summary, list('Phone Usage','onehot',nrow(df_lrt_day_significant),nrow(df_lrt_hour_significant),nrow(df_lrt),nrow(df_day_failed),nrow(df_hour_failed)))
# Per participant
for (ID in unique(df_lrt$ID)) {
    if (nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_likelihood<.05,]) > 0) {
        print(ID)
        print(nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_likelihood<.05,]))
    }
}


In [ ]:
%%R
# Phone Usage - sin cos
df_lrt <- read_saved_lrts("Sin Cos Hour Day", "Phone Usage", "_sincos_transform")

df_lrt <- df_lrt[complete.cases(df_lrt[ ,1]),]  # There are some NA rows
df_lrt$Day_likelihood <- as.numeric(as.character(df_lrt$Day_likelihood))
df_lrt$Hour_likelihood <- as.numeric(as.character(df_lrt$Hour_likelihood))

df_lrt_day_significant <- subset(df_lrt, Day_likelihood<.05)
df_lrt_hour_significant <- subset(df_lrt, Hour_likelihood<.05)
df_day_failed <- subset(df_lrt, Day_likelihood=="Failed")  # If want to check this have to remove as.numeric()
df_hour_failed <- subset(df_lrt, Hour_likelihood=="Failed")
df_lrt_summary <- rbind(df_lrt_summary, list('Phone Usage','sincos',nrow(df_lrt_day_significant),nrow(df_lrt_hour_significant),nrow(df_lrt),nrow(df_day_failed),nrow(df_hour_failed)))
# Per participant
for (ID in unique(df_lrt$ID)) {
    if (nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_likelihood<.05,]) > 0) {
        print(ID)
        print(nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_likelihood<.05,]))
    }
}

In [ ]:
%%R
# Hour + day models
df_lrt <- read_saved_lrts("One Hot Hour Day In Same Model", "Phone Usage", "_day-hour-to-hour_onehot_transform")

df_lrt <- df_lrt[complete.cases(df_lrt[ ,1]),]  # As there are some completely NA rows
df_lrt$Day_hour_likelihood <- as.numeric(as.character(df_lrt$Day_hour_likelihood))
df_lrt_day_hour_significant <- subset(df_lrt, Day_hour_likelihood<.05)
df_day_hour_failed <- subset(df_lrt, Day_hour_likelihood=="Failed")  # If want to check this have to remove as.numeric()
df_lrt_summary_day_hour_combined <- rbind(df_lrt_summary_day_hour_combined, list('Phone Usage','onehot',nrow(df_lrt_day_hour_significant),nrow(df_day_hour_failed)))
# Per participant
for (ID in unique(df_lrt$ID)) {
    print(ID)
    print(nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_hour_likelihood<.05,]))
}

df_lrt <- read_saved_lrts("Sin Cos Hour Day In Same Model", "Phone Usage", "_day-hour-to-hour_sincos_transform")

df_lrt <- df_lrt[complete.cases(df_lrt[ ,1]),]  # As there are some completely NA rows
df_lrt$Day_hour_likelihood <- as.numeric(as.character(df_lrt$Day_hour_likelihood))
df_lrt_day_hour_significant <- subset(df_lrt, Day_hour_likelihood<.05)
df_day_hour_failed <- subset(df_lrt, Day_hour_likelihood=="Failed")  # If want to check this have to remove as.numeric() 
df_lrt_summary_day_hour_combined <- rbind(df_lrt_summary_day_hour_combined, list('Phone Usage','sincos',nrow(df_lrt_day_hour_significant),nrow(df_day_hour_failed)))
# Per participant
for (ID in unique(df_lrt$ID)) {
    print(ID)
    print(nrow(df_lrt[df_lrt$ID==ID & df_lrt$Day_hour_likelihood<.05,]))
}

### Compare out-of-sample model likelihoods

In [ ]:
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='onehot')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
#Onehot
df_model_parameters <- read_saved_models(folder="One Hot Hour Day", "Phone Usage", "_onehot_transform")
simulation_model_hour_onehot = simulate_points(channel='Phone Usage', model_type="model_hour_onehot", dataframe=df_ID_app, IDs=BEHAPP_ID_list, basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)
simulation_model_day_onehot = simulate_points(channel='Phone Usage', model_type="model_day_onehot", dataframe=df_ID_app, IDs=BEHAPP_ID_list, basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)


In [ ]:
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='sin_cos')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
#Sin cos
df_model_parameters <- read_saved_models(folder="Sin Cos Hour Day", "Phone Usage", "_sincos_transform")
simulation_model_hour_sincos = simulate_points(channel='Phone Usage', model_type="model_hour_sincos", dataframe=df_ID_app, IDs=BEHAPP_ID_list, basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)
simulation_model_day_sincos = simulate_points(channel='Phone Usage', model_type="model_day_sincos", dataframe=df_ID_app, IDs=BEHAPP_ID_list, basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)


In [ ]:
df_ID_app = df[['ID','weekday','hour','Phone Usage','Mask']].copy()

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
# Homogeneous
df_model_parameters <- read_saved_models(folder="Homogeneous Poisson", "Phone Usage", "_homogeneous")
simulation_model_homog = simulate_points_baselines(channel='Phone Usage', model_type="model_homog", dataframe=df_ID_app, IDs=BEHAPP_ID_list)


In [ ]:
%%R
# Phone Usage
all_out_of_sample_likelihoods <- data.frame(ID = simulation_model_day_onehot$ID_list, Fold = simulation_model_day_onehot$fold_list, Homogeneous = unlist(simulation_model_homog$out_of_sample_likelihood), 'Hour onehot' = unlist(simulation_model_hour_onehot$out_of_sample_likelihood), 'Hour sin cos' = unlist(simulation_model_hour_sincos$out_of_sample_likelihood), 'Day onehot' = unlist(simulation_model_day_onehot$out_of_sample_likelihood), 'Day sincos' = unlist(simulation_model_day_sincos$out_of_sample_likelihood))
all_out_of_sample_likelihoods_mean <- aggregate(list(Homogeneous = all_out_of_sample_likelihoods$Homogeneous, Hour.onehot = all_out_of_sample_likelihoods$Hour.onehot, Hour.sin.cos = all_out_of_sample_likelihoods$Hour.sin.cos, Day.onehot = all_out_of_sample_likelihoods$Day.onehot, Day.sincos = all_out_of_sample_likelihoods$Day.sincos), by=list(all_out_of_sample_likelihoods$ID), FUN=mean, na.rm=TRUE)
all_out_of_sample_likelihoods_mean


#### Check oos likelihood for subset of participants where day+hour models were significant in LRTs

In [ ]:
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='onehot')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
#Onehot
df_model_parameters <- read_saved_models(folder="One Hot Hour Day In Same Model", "Phone Usage", "_day-hour-subset_onehot_transform")
df_ID_app <- subset(df_ID_app, ID %in% df_model_parameters$ID)
simulation_model_day_hour_onehot = simulate_points(channel='Phone Usage', model_type="model_day_hour_onehot", dataframe=df_ID_app, IDs=unique(df_model_parameters$ID), basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)


In [ ]:
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='sin_cos')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
#Sin cos
df_model_parameters <- read_saved_models(folder="Sin Cos Hour Day In Same Model", "Phone Usage", "_day-hour-subset_sincos_transform")
df_ID_app <- subset(df_ID_app, ID %in% df_model_parameters$ID)
simulation_model_day_hour_sincos = simulate_points(channel='Phone Usage', model_type="model_day_hour_sincos", dataframe=df_ID_app, IDs=unique(df_model_parameters$ID), basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)


In [ ]:
%%R
out_of_sample_likelihoods_onehot <- data.frame(ID = simulation_model_day_hour_onehot$ID_list, Fold = simulation_model_day_hour_onehot$fold_list, 'Day Hour onehot' = unlist(simulation_model_day_hour_onehot$out_of_sample_likelihood))
out_of_sample_likelihoods_sincos <- data.frame(ID = simulation_model_day_hour_sincos$ID_list, Fold = simulation_model_day_hour_sincos$fold_list, 'Day Hour sincos' = unlist(simulation_model_day_hour_sincos$out_of_sample_likelihood))
out_of_sample_likelihoods_onehot_mean <- aggregate(list(Day.Hour.onehot = out_of_sample_likelihoods_onehot$Day.Hour.onehot), by=list(out_of_sample_likelihoods_onehot$ID), FUN=mean, na.rm=TRUE)
out_of_sample_likelihoods_sincos_mean <- aggregate(list(Day.Hour.sincos = out_of_sample_likelihoods_sincos$Day.Hour.sincos), by=list(out_of_sample_likelihoods_sincos$ID), FUN=mean, na.rm=TRUE)


### Simulate masked periods

In [ ]:
df_ID_app = encode_covariates(dataframe=df, channel='Phone Usage', encode='onehot')

In [ ]:
%%R -i df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle

df_model_parameters <- read_saved_models(folder="One Hot Hour Day", "Phone Usage", "_onehot_transform")
simulation_model_hour = simulate_points(channel='Phone Usage', model_type="model_hour_onehot", dataframe=df_ID_app, IDs=unique(df_model_parameters$ID), basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)


In [ ]:
channel = 'Phone Usage'
transform = 'Onehot Hour '

imputed_channel = transform + 'Imputed ' + channel
columns = list(df_original.columns) + [imputed_channel]
df_with_imputed = pd.DataFrame(columns=columns) # df to add imputed time series to

simulated_timestamps_series_model_hour = r['simulation_model_hour'][2]
simulated_timestamps_series_model_hour = [[np.array(simulated_timestamps_series_model_hour[x])] for x in range(len(simulated_timestamps_series_model_hour))]

simulated_timestamps=simulated_timestamps_series_model_hour
cross_validation_ID_list=r['simulation_model_hour'][4]
fold_list=r['simulation_model_hour'][3]

df_simulated_timestamps = pd.DataFrame({'ID': cross_validation_ID_list, 'simulated_timestamps': simulated_timestamps, 'Fold': fold_list})
ID_list = list(set(cross_validation_ID_list))
# For each participant:
for ID in ID_list:
    df_current = df_ID_app.loc[df_ID_app['ID'] == ID]
    df_original_current = df_original.loc[df_original['ID'] == ID]
    for fold in range(1,11):
        df_real_masked_sections = df_original_current.drop(df_original_current[df_original_current['Mask'] != fold].index) # Need to identify each of the masked periods
        df_real_masked_sections = df_real_masked_sections.sort_index().reset_index(drop=True)
        df_real_masked_sections[imputed_channel] = 1
        imputed_series = df_simulated_timestamps.loc[(df_simulated_timestamps['ID']==ID) & (df_simulated_timestamps['Fold']==fold), 'simulated_timestamps'].item()      
        if np.isnan(imputed_series[0][0]) == False:  # Check that activity was actually simulated
            df_real_masked_sections.loc[imputed_series[0]-1,imputed_channel] = 2  # Take 1 away from R list to index from 0
        df_with_imputed = pd.concat([df_with_imputed, df_real_masked_sections], ignore_index=True)
 

### Visualisation example

In [ ]:
channel = "Phone Usage"
df_ID_app = encode_covariates(dataframe=df, channel=channel, encode='onehot')

In [ ]:
%%R -i channel,df_ID_app,BEHAPP_ID_list,basic_sin_day_cycle,basic_cos_day_cycle,basic_sin_hour_cycle,basic_cos_hour_cycle,basic_onehot_day_matrix_cycle,basic_onehot_hour_matrix_cycle
#Onehot hour
df_model_parameters <- read_saved_models(folder="One Hot Hour Day", channel, "_onehot_transform")
simulation_model_hour_onehot = simulate_points(channel=channel, model_type="model_hour_onehot", dataframe=df_ID_app, IDs=list('SMARD_117037/SMARD_6ZHBZ8','SMARD_117064','SMARD_116862'), basic_sin_day_cycle, basic_cos_day_cycle, basic_sin_hour_cycle, basic_cos_hour_cycle, basic_onehot_day_matrix_cycle, basic_onehot_hour_matrix_cycle)
#Homog
df_model_parameters <- read_saved_models(folder="Homogeneous Poisson", channel, "_homogeneous")
simulation_model_homog = simulate_points_baselines(channel=channel, model_type="model_homog", dataframe=df_ID_app, IDs=list('SMARD_117037/SMARD_6ZHBZ8','SMARD_117064','SMARD_116862'))


In [ ]:
# Include homogeneous as well as hour models, and combine figures for different participants (just one fold for each participant):
plt.rcParams.update({'font.size': 20})
# Hour model:
simulated_timestamps_series_model_hour = r['simulation_model_hour_onehot'][2]  
simulated_timestamps_series_model_hour = [[np.array(simulated_timestamps_series_model_hour[x])] for x in range(len(simulated_timestamps_series_model_hour))]
# Homogeneous model:
simulated_timestamps_series_model_homog = r['simulation_model_homog'][2]
simulated_timestamps_series_model_homog = [[np.array(simulated_timestamps_series_model_homog[x])] for x in range(len(simulated_timestamps_series_model_homog))]

simulated_timestamps=simulated_timestamps_series_model_hour
simulated_timestamps_homog=simulated_timestamps_series_model_homog
cross_validation_ID_list=['a','b','c']*10  # Plot for chosen IDs
fold_list=[1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6,7,7,7,8,8,8,9,9,9,10,10,10]

df_simulated_timestamps = pd.DataFrame({'ID': cross_validation_ID_list, 'simulated_timestamps': simulated_timestamps, 'simulated_timestamps_homog': simulated_timestamps_homog,'Fold': fold_list})
# For each participant:
n = 0 # Index for subplot
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(20, 8))
fig.tight_layout(h_pad=1.5, w_pad=-0.7)
for ID in ['a','b','c']:  # Plot for chosen IDs
    df_current = df_ID_app.loc[df_ID_app['ID'] == ID]
    df_original_current = df_original.loc[df_original['ID'] == ID]
    for fold in [1]:  # Just do one fold
        df_real_masked_sections = df_original_current.drop(df_original_current[df_original_current['Mask'] != fold].index) # Identify each of the masked periods
        df_real_masked_sections['hour'] = pd.to_datetime(df_real_masked_sections['hour'], format='%Y-%m-%d %H:%M:%S').dt.hour
        for i in df_real_masked_sections.index[:-1]:
            if (abs(df_real_masked_sections['hour'][i+1] - df_real_masked_sections['hour'][i]) > 1) & (df_real_masked_sections['hour'][i] != 23):
                hours_needed = range(df_real_masked_sections['hour'][i]+1, 24)
                new_index_list = np.linspace(i+0.1, i+0.9,len(hours_needed)) 
                hour_index = 0
                for i in new_index_list:
                    df_real_masked_sections.loc[i] = i, ID, 0, 0, list(hours_needed)[hour_index], 0, 0, 0, np.nan, 0, 0, 0, np.nan, np.nan, np.nan, np.nan, np.nan, 0, 0, np.nan, 'SMARD', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan                
                    hour_index = hour_index + 1
        df_real_masked_sections = df_real_masked_sections.sort_index().reset_index(drop=True)
        real_series = np.array(df_real_masked_sections[channel])
        imputed_series = df_simulated_timestamps.loc[(df_simulated_timestamps['ID']==ID) & (df_simulated_timestamps['Fold']==fold), 'simulated_timestamps'].item()      
        imputed_series_homog = df_simulated_timestamps.loc[(df_simulated_timestamps['ID']==ID) & (df_simulated_timestamps['Fold']==fold), 'simulated_timestamps_homog'].item()              
        # Use timestamps to assign activity values for whole time series
        imputed_series2 = [1] * len(real_series)
        imputed_series_homog2 = [1] * len(real_series)
        if 2 in real_series:
            for i in imputed_series[0]:
                imputed_series2[int(i)-1] = 2  # Take away 1 as R starts from 1 not 0  
            for i in imputed_series_homog[0]:
                imputed_series_homog2[int(i)-1] = 2  # Take away 1 as R starts from 1 not 0  
        # Pad the time series (at start and end) so grid has hour 0-23
        series_hours = df_real_masked_sections['hour']
        start_hour = int(series_hours[0])
        if start_hour != 0:
            real_series = np.concatenate([[0] * start_hour, real_series])
            imputed_series2 = np.concatenate([[0] * start_hour, imputed_series2])
            imputed_series_homog2 = np.concatenate([[0] * start_hour, imputed_series_homog2])
        end_hour = int(series_hours[len(series_hours)-1])
        end_gap = 23 - end_hour
        if end_hour != 23:
            real_series = np.concatenate([real_series, [0] * end_gap])
            imputed_series2 = np.concatenate([imputed_series2, [0] * end_gap])
            imputed_series_homog2 = np.concatenate([imputed_series_homog2, [0] * end_gap])
        real_grid = np.reshape(real_series, (int(len(real_series)/24), 24))
        imputed_grid = np.reshape(imputed_series2, (int(len(imputed_series2)/24), 24))
        imputed_grid_homog = np.reshape(imputed_series_homog2, (int(len(imputed_series_homog2)/24), 24))
        real_plot = ax[n,0].imshow(real_grid, cmap=colors.ListedColormap(['grey','powderblue', 'orchid']), aspect='auto',
                  interpolation='nearest')  # Fixes blurring  
        imputed_plot = ax[n,1].imshow(imputed_grid, cmap=colors.ListedColormap(['grey','powderblue', 'orchid']), aspect='auto',
                  interpolation='nearest')
        imputed_plot_homog = ax[n,2].imshow(imputed_grid_homog, cmap=colors.ListedColormap(['grey','powderblue', 'orchid']), aspect='auto',
                  interpolation='nearest')
        ax[n,0].set_ylabel('Day', labelpad=10)
        n = n+1  # Update index for next participant

ax[0,0].set_title('a)',loc='left') 
ax[1,0].set_title('b)',loc='left')
ax[2,0].set_title('c)',loc='left')

ax[2,0].set_xlabel('Hour')
ax[2,1].set_xlabel('Hour')
ax[2,2].set_xlabel('Hour')
ax[0,0].set_title('Real')
ax[0,1].set_title('Hour model')
ax[0,2].set_title('Homogeneous model')

for i in [0,1]:
    for j in [0,1,2]:
        ax[i,j].set_xticks([])  # Remove x-axis ticks
for i in [0,1,2]:
    for j in [1,2]:
        ax[i,j].set_yticks([])  # Remove y-axis ticks
    
real_colorbar = plt.colorbar(imputed_plot, ax=ax, shrink=0.35, location='bottom')
real_loc = [0.4,1,1.6]
real_colorbar.set_ticks(real_loc)  
real_labels = ['Missing','No activity', 'Activity']
real_colorbar.set_ticklabels(real_labels) 

plt.savefig('/directory/figure_name.png', bbox_inches='tight')
plt.show()